In [1]:
# 8.2.1 Extract the Wikipedia Movies JSON

import json
import pandas as pd
import numpy as np

In [2]:
file_dir = '/Users/darrenlemyacrawford/Desktop/Class folder/Movies-ETL/wikipedia-movies.json'

In [3]:
f'{file_dir}filename'

'/Users/darrenlemyacrawford/Desktop/Class folder/Movies-ETL/wikipedia-movies.jsonfilename'

You may be tempted to try to read the JSON files directly into a Pandas DataFrame. While technically that may be possible, the read_json method that comes built into the Pandas library only works well for data that is already clean—for example, when the JSON data has every field filled in every time it is returned. We call data like this "flat."

The type of data we get from doing a scrape of Wikipedia is pretty messy, so it's easier to load the raw JSON as a list of dictionaries before converting it to a DataFrame.

# Load the JSON into a List of Dictionaries

### To load the raw JSON into a list of dictionaries, we will use the load() method

When opening files in Python, we want to use the "with" statement to handle the file resource.

Using the "with" statement, open the Wikipedia JSON file to be read into the variable "file", and use "json.load()" to save the data to a new variable.

In [4]:
with open(file_dir, mode='r') as file:
    wiki_movies_raw = json.load(file)

In [5]:
len(wiki_movies_raw)

7311

Is 7,311 a reasonable number of records? We just want to make sure that we don't have an outlandishly large or small number. If we do, there's potentially something seriously wrong with the data that needs to be investigated before moving on.

One way to check that 7,311 is reasonable is to look at the rate of movies being released that it implies. Rough mental math here is the key—we want these calculations to be quick enough that these checks become a habit. So, let's say that it's about 7,200 movies over about 30 years. That's 240 movies released per year, or a little less than five movies released per week. That seems a little high if we're considering only major movies, but if for every two major motion pictures, there are three indie films, that doesn't seem like an outlandish number.

- always take a look at a few individual records just to make sure data isn't garbled... With a DataFrame, we'd do this with the "head()" and "tail()" methods, **but with a list of dictionaries, we need to inspect the records directly.

In [6]:
# To see the First 5 records

wiki_movies_raw[:5]

[{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane',
  'year': 1990,
  'imdb_link': 'https://www.imdb.com/title/tt0098987/',
  'title': 'The Adventures of Ford Fairlane',
  'Directed by': 'Renny Harlin',
  'Produced by': ['Steve Perry', 'Joel Silver'],
  'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'],
  'Story by': ['David Arnott', 'James Cappe'],
  'Based on': ['Characters', 'by Rex Weiner'],
  'Starring': ['Andrew Dice Clay',
   'Wayne Newton',
   'Priscilla Presley',
   'Lauren Holly',
   'Morris Day',
   'Robert Englund',
   "Ed O'Neill"],
  'Narrated by': 'Andrew "Dice" Clay',
  'Music by': ['Cliff Eidelman', 'Yello'],
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Michael Tronick',
  'Productioncompany ': 'Silver Pictures',
  'Distributed by': '20th Century Fox',
  'Release date': ['July 11, 1990', '(', '1990-07-11', ')'],
  'Running time': '102 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$20 million',


In [7]:
# To see the Last 5 records

wiki_movies_raw[-5:]

[{'url': 'https://en.wikipedia.org/wiki/Holmes_%26_Watson',
  'year': 2018,
  'imdb_link': 'https://www.imdb.com/title/tt1255919/',
  'title': 'Holmes & Watson',
  'Directed by': 'Etan Cohen',
  'Produced by': ['Will Ferrell',
   'Adam McKay',
   'Jimmy Miller',
   'Clayton Townsend'],
  'Screenplay by': 'Etan Cohen',
  'Based on': ['Sherlock Holmes',
   'and',
   'Dr. Watson',
   'by',
   'Sir Arthur Conan Doyle'],
  'Starring': ['Will Ferrell',
   'John C. Reilly',
   'Rebecca Hall',
   'Rob Brydon',
   'Steve Coogan',
   'Ralph Fiennes'],
  'Music by': 'Mark Mothersbaugh',
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Dean Zimmerman',
  'Productioncompanies ': ['Columbia Pictures',
   'Gary Sanchez Productions',
   'Mosaic Media Group',
   'Mimran Schur Pictures'],
  'Distributed by': 'Sony Pictures Releasing',
  'Release date': ['December 25, 2018',
   '(',
   '2018-12-25',
   ')',
   '(United States)'],
  'Running time': '90 minutes',
  'Country': 'United States',
  'Language

- It's always a good idea to check records in the middle as well. Choose a number somewhere around the halfway mark and look at a handful of records after that index.

If everything looks good, congratulations! You're halfway through the Extract step. Now we'll load in the Kaggle data.

In [8]:
# Some records in the middle

wiki_movies_raw[3600:3605]

[{'url': 'https://en.wikipedia.org/wiki/Benji:_Off_the_Leash!',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0315273/',
  'title': 'Benji: Off the Leash!',
  'Directed by': 'Joe Camp',
  'Written by': 'Joe Camp',
  'Starring': ['Benji', 'Nick Whitaker', 'Shaggy', 'Gypsy the Cockatoo'],
  'Music by': 'Antonio di Lorenzo',
  'Productioncompany ': 'Mulberry Square Productions',
  'Distributed by': 'Mulberry Square Productions',
  'Release date': ['March 26, 2004', '(', '2004-03-26', ')'],
  'Running time': '97 min',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$3,817,362'},
 {'url': 'https://en.wikipedia.org/wiki/The_Best_Thief_in_the_World',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0389796/',
  'title': 'The Best Thief in the World',
  'Directed by': 'Jacob Kornbluth',
  'Produced by': ['Tim Perrell', 'Nicola Usborne'],
  'Written by': 'Jacob Kornbluth',
  'Starring': ['Marc Rozendaal',
   'Michael Silverman',
   'David Warsh

# 8.2.2 Extract the Kaggle Data

The Kaggle dataset pulls from the MovieLens dataset of over 20 million reviews and contains a metadata file with details about the movies from The Movie Database (TMDb) (Links to an external site.). Download the zip file from Kaggle (Links to an external site.), extract it to your class folder, and decompress the CSV files. We're interested in the movies_metadata.csv and ratings.csv files.

In [9]:
# Since the Kaggle data is already in flat-file formats, we'll just pull them into 
# Pandas DataFrames directly with the following code:

kaggle_metadata = pd.read_csv('movies_metadata.csv', low_memory=False)
ratings = pd.read_csv('ratings.csv')

"low_memory=False" w/o this there will be a warning about a column with mixed data types

In [10]:
kaggle_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [11]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


- Errors can still occur in the middle of the file, so the best practice is to sample a handful of rows randomly using the "sample()" method. For a DataFrame called "df", "df.sample(n=5)" will show five random rows from the dataset.

### You've just completed the "Extract" step in ETL. We're going to use many of the same ideas, when we get to "Load", we're going to use many of the same ideas we just used to extract the data.

# 8.3.1 Data-Cleaning Strategies
## (Transform: Clean Individual Datasets)

Wikipedia doesn't have strict standards on how movie data is presented, so it needs a lot of work to clean up the data and make it usable, it's in the flexible JSON format, needs to be organized in a structured format before it can be sent to SQL.

Explore your options for cleaning the dataset...The transform step is largely spent on data cleaning. 
Plan. Inspect. Execute.

#### Data beyond repair

could be data that has been overwritten or has suffered severe data corruption during storage or transfer (such as power loss during writing, voltage spikes, or hard-drive failures). The worst-case example would be having data with every value missing. All the information is lost and unrecoverable. For data beyond repair, all we can do is delete it and move on.

#### Data that is badly damaged

may have good data that we can recover, but it will take time and effort to repair the damaged data. This can be garbled data, with a lot of missing values, from inconsistent sources, or existing in multiple columns. Consider trade-offs to pick the best solution (even if the "best" solution isn't perfect, but rather the "best-available" solution). To repair badly damaged data, try these strategies:

- Filling in missing data by
    - substituting data from another source,
    - interpolating between existing data points, or
    - extrapolating from existing data
- Standardizing units of measure (e.g., monetary values stored in multiple currencies)
- Consolidating data from multiple columns

#### Data in the wrong form

should usually be fixed—that is, the data is good but can't be used in its current form. "Good" data in the wrong form can be data that is too granular or detailed, numeric data stored as strings, or data that needs to be split into multiple columns (e.g., address data). To remedy good data in the wrong form, try these strategies:

- Reshape the data
- Convert data types
- Parse text data to the correct format
- Split columns

There is no simple checklist or flowchart we can use to guide; In data cleaning, one has to constantly ask what we might've missed, and following a rigid plan means we won't be asking ourselves those important questions. Data cleaning requires a lot of improvising.

It's important to *document your data cleaning assumptions as well as decisions and their motivations.

As you clean one part of the data, you may reveal something messy in another part, that means *unwinding a lot of work that you've already done and having to redo it with a slight change.

We're not going to try and clean all the data at once. Instead, we're going to focus on *one problem at a time using an iterative process.

# 8.3.2 Iterative Process for Cleaning Data
## (Transform: Clean Individual Datasets)
#### Inspect Plan Execute

Early iterations focus on making the data easier to investigate: deleting obviously bad data, removing superfluous columns (e.g., columns with only one value or missing an overwhelming amount of data), removing duplicate rows, consolidating columns, and reshaping the data if necessary.

...more subtle problems become noticeable.

It's rare to reach a point where no more problems exist in the data. More likely, a point is reached where the work to fix any remaining problems isn't worth the amount of data that would be recovered. After the remaining issues are documented, the transform step is considered finished.

#### Inspect

Most usable data contains too many data points to review every single one, so we'll need to use strategies that tell us about the whole dataset.

First, count how many data points or rows exist. If the data is structured, count the number of columns and missing values in each column. If possible, count the number of unique values in each column and how frequently each unique value appears.

If a column's data type is numeric, we can summarize its data with some basic statistics, such as measures of central tendency (e.g., mean and/or median) and measures of spread (e.g., standard deviation, interquartile range, minimum/maximum). We can also investigate columns with statistical plots, like scatter plots and histograms.

#### Plan

After we've investigated our data and started to identify problem areas, we can make decisions about how to fix the problems...articulating the problems clearly...we'll answer several questions, including:

- If a column doesn't have the right data type, is it a problem with the whole column? Or are just a handful of rows causing the issues?
- Do rows have outliers due to spurious data? Or are they valid data points?
- When values are missing, will they need to be removed, replaced, or interpolated?

The answers to these questions will tell us how we need to modify our data. Keep in mind, there are two main ways: 1)we can modify values and 2)we can modify structure. Modifying data values includes removing rows or columns, replacing values, or generating new columns from old ones.

* Converting a column to a new data type is also a form of replacing values. We can also bin data (like rounding to the nearest hundred), replacing numeric data (e.g., income) with categorical data (e.g., income brackets). We might generate new columns by splitting an existing column into several new columns—by splitting an address column to street, city, state, and zip code columns, for example—or by calculating a new column from multiple existing columns, like calculating total price by multiplying item prices by quantities.

With clearly stated steps to fix the problem, we can make an informed decision about whether implementing the plan is worth the effort. Sometimes there are multiple viable resolutions to choose from. To decide, we weigh trade-offs and ultimately choose the best option.

#### Execute

As we write code to fix the problem, we might discover that the problem is more difficult than initially expected. This is a normal part of the process. As you implement your changes, try to take into account any unintended consequences you could introduce.

##### Cleaning up messy data is a messy process. The best practice is to document every step of your thought process and actions in detail.

# 8.3.3 Investigate the Wikipedia Data
## (Transform: Clean Individual Datasets)

*Initial Investigation - One of the easiest ways to find glaring errors is to just pretend as if there aren't any, and try to jump straight to the finish line. Eventually, we want to clean up the Wikipedia data into tabular data with rows and columns, so let's see what happens if we create a DataFrame from our raw data.

In [12]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
wiki_movies_df.head()

#### That's a lot of columns! Even if we try to use "print(wiki_movies_df.columns)", they won't all print out. We'll have to convert "wiki_movies_df.columns" to a list to see all of the columns.

#### Use "wiki_movies_df.columns.tolist()" and run the cell to see all of the column names that were imported.

In [14]:
wiki_movies_df.columns.tolist()

#  We can identify column names that don't relate to movie data, such as "Dewey Decimal," 
#  "Headquarters," and "Number of employees.

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti

*Plan - Let's modify our JSON data by restricting it to only those entries that have a director and an IMDb link. We can do this with a list comprehension.

## Use List Comprehensions to Filter Data
We've used list comprehensions previously as a compact way to apply a function to every element in a list.

So far, we've used list comprehensions in the form to compress code that would have been done in a for loop.

#### [expression for element in source_list]

We can also filter out results using a conditional filter expression, as shown below:

#### [expression for element in source_list if filter_expression]

The resulting list will only have elements where the filter expression evaluates to True.

To create a filter expression for only movies with a director and an IMDb link, keep in mind that there are two columns in the data for director information. We'll need to check if either "Director" or "Directed by" are keys in the current dict. If there is a director listed, we also want to check that the dict has an IMDb link. Luckily, that information is only in one column, imdb_link, so our filter expression will look like the following:

#### if ('Director' in movie or 'Directed by' in movie) and 'imdb_link' in movie

*Execute - Create a list comprehension with the filter expression we created and save that to an intermediate variable wiki_movies. See how many movies are in wiki_movies with the len() function.

In [15]:
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie]
len(wiki_movies)

7080

This only cuts the number of movies down to 7,080. Not too bad. Make a DataFrame from "wiki_movies", and there should only be 78 columns. It may seem counterintuitive that we have fewer columns in the result set when we constrain our results to rows that must have data in certain columns. But there may be columns that only apply to rows that do not have data in the columns we’re targeting. In those cases, the columns will be eliminated, as seen here.

#### This is why it's easier to load the JSON in first and then convert it to a DataFrame. Instead of trying to identify which columns in our DataFrame don't belong, we just remove the bad data points, and the bad columns never get imported in.

In [16]:
wiki_movies_df = pd.DataFrame(wiki_movies)
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


*Inspect - 78 columns are still a lot of columns, so let's keep investigating. 

One thing to watch out for is to make nondestructive edits as much as possible while designing your pipeline. That means it's better to keep your raw data in one variable, and put the cleaned data in another variable. It takes up more memory, but it makes tracking the iterative process of data cleaning easier.

#### For example, if we had just deleted the movies from wiki_movies_raw that didn't have "Directed by" as a key, we'd have made a destructive edit and missed that some have "Director" as the key

*Inspect - There sure are a lot of languages—we'll get to those shortly. For now, one of the columns that stands out is "No. of episodes."

*Plan - It looks like we've got some TV shows in our data instead of movies. We'll want to get rid of those, too.

*Execute - Don't worry if you didn't catch the "No. of episodes" column in the list. Cleaning data is an iterative process, and if you started with cleaning up the language data first, or some other part of the data, you would see the "No. of episodes" column soon enough. The key is to keep reworking the pipeline bit by bit.

We'll add that filter to our list comprehension:

In [33]:
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]
len(wiki_movies)

7076

# 8.3.4 Revisit Functions
## (Transform: Clean Individual Datasets)

Now that you've filtered out bad data, you need to clean up each movie entry so it's in a standard format. If you can make one process broad enough to handle every movie entry, you can apply that process repeatedly for every movie entry. For this task you will create a function.

#### To keep things organized, we are going to make a function that performs that cleaning process.

Functions are blocks of code within a script or algorithm that perform a specific task. There are four basic parts to a function:

    - Name
    - Parameters
    - Code block
    - Return value
    
Scope - Inside the code block of a function, we can use variables that were created outside the function and initialize new variables inside the function.
This is called the "scope" of the variables:

- Variables created outside the function are called "global variables".
- New variables created inside the function are "local variables".
- The hierarchy of variables is called the scope.

### Lambda Functions
There's a special function we can make in Python called lambda, which is the most stripped-down kind we can make.

Lambda functions are written in one line and automatically return a value without using the return keyword. Lambda functions have no name and are also known as "anonymous functions." There are functions that expect other functions to be sent to them as a parameter, and lambda functions are a way to quickly create a concise function to send as a parameter to another function...the basic syntax to a lambda function follows:
#### lambda arguments: expression
This function will take in an argument and will return the expression. Even though lambda functions are supposed to be anonymous, just this once we'll create a lambda function and assign a name so that we can see how they work. A lambda function that squares a value looks like the following:
#### lambda x: x * x
Here, x is the argument, and x * x is the expression. Let's assign this to a name so that we can use it:
#### square = lambda x: x * x
#### square(5)
The output will be:
#### 25

#### If this seems strange, don't worry. The main benefit of lambda functions is that they can be used as one-time-use functions. We'll talk more about lambda functions when we have a more natural use case for them—they'll make more sense at that time.

# 8.3.5 Create a Function to Clean the Data, Part 1
## (Transform: Clean Individual Datasets)

Filtering out bad data isn't enough. You know that you need to make sure the good data that you have is clean enough to use...Now we're ready to create our function to clean our movie data.

First, write a simple function to make a copy of the movie and return it. As we work with our data, we'll iteratively add more to our code block. To start, call the function "clean_movie", and have it take "movie" as a parameter.
#### def clean_movie(movie):
Because the movies are dicts and we want to make nondestructive edits, make a copy of the incoming movie. To make a copy of "movie", we'll use the "dict()" constructor.
- Constructors are special functions that initialize new objects. They reserve space in memory for the object and perform any initializations the object requires. Also, constructors can take parameters and initialize a new object using those parameters.

When we pass "movie" as a parameter to the "dict()" constructor, it reserves a new space in memory and copies all of the info in "movie" to that new space:

In [18]:
def clean_movie(movie):
    movie_copy = dict(movie)

However, we have another trick that's even better.

Inside of the function, we can create a new local variable called "movie" and assign it the new copy of the parameter "movie".

In [19]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy

#### This way, inside of the "clean_movie()" function, "movie" will refer to the local copy. Any changes we make inside "clean_movie()" will now only affect the copy, so if we make a mistake, we still have the original, untouched "movie" to reference.

To finish our skeleton of the "clean_movie" function, return the "movie" variable. (This function doesn't do much right now, but we'll be adding more to it soon.)

In [20]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    return movie

*Inspect - take a look at what's going on with those languages. The first one on the list is Arabic, so let's see which movies have a value for "Arabic."

In [21]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]

# The different language columns are for alternate titles of the movie.

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
6838,https://en.wikipedia.org/wiki/The_Insult_(film),2018,https://www.imdb.com/title/tt7048622/,The Insult,Ziad Doueiri,"[Rachid Bouchareb, Jean Bréhat, Julie Gayet, A...",NaN,NaN,NaN,"[Adel Karam, Kamel El Basha]",...,NaN,Case No. 23,NaN,NaN,NaN,قضية رقم ٢٣,Qadiyya raqm 23,NaN,NaN,NaN
7062,https://en.wikipedia.org/wiki/Capernaum_(film),2018,https://www.imdb.com/title/tt8267604/,Capernaum,Nadine Labaki,"[Michel Merkt, Khaled Mouzanar]","[Nadine Labaki, Jihad Hojaily, Michelle Keserw...","[Georges Khabbaz, Nadine Labaki, Michelle Kese...",NaN,"[Zain Al Rafeea, Yordanos Shiferaw, Boluwatife...",...,NaN,NaN,NaN,NaN,NaN,کفرناحوم‎,NaN,NaN,NaN,NaN


In [22]:
# Visit the movie's Wikipedia page ...for more details.

wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

6838    https://en.wikipedia.org/wiki/The_Insult_(film)
7062     https://en.wikipedia.org/wiki/Capernaum_(film)
Name: url, dtype: object

*Plan - The different language columns are for alternate titles of the movie. Let's combine all of them into one dictionary that has all the alternate titles.

To do that, we need to go through each of the columns, one by one, and determine which are alternate titles. Some might be tricky. If you're not sure what a column name means, google it. Also, review a column's data to understand the type of content in that column.

For example, perhaps you've never heard of "McCune–Reischauer." Is it an esoteric filmmaking technique? Google it, and you'll learn it's a romanization system for Korean. Now look at the actual values contained in the column. If the values don't make sense to you either, google them, too.

#### The "value_counts()" method is a quick, easy way to see what non-null values there are in a column.

In [23]:
wiki_movies_df['McCune–Reischauer'].notnull().value_counts()

False    7078
True        2
Name: McCune–Reischauer, dtype: int64

In [24]:
wiki_movies_df['Arabic'].notnull().value_counts()

False    7078
True        2
Name: Arabic, dtype: int64

### SKILL DRILL
Go through each of the columns, one by one, and determine which columns hold alternate titles.

Hint: You might find it easier to sort the column names first as you're going through them. The following will display columns in alphabetical order.
#### sorted(wiki_movies_df.columns.tolist())
**a complete list of columns that hold alternate title data: Also known as, Arabic, Cantonese, Chinese, French, Hangul, Hebrew, Hepburn, Japanese, Literally,  Mandarin, McCune–Reischauer, Original title, Polish,  Revised Romanization, Romanized, Russian, Simplified, Traditional, Yiddish

In [25]:
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Also known as',
 'Animation by',
 'Arabic',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Camera setup',
 'Cantonese',
 'Chinese',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'French',
 'Genre',
 'Hangul',
 'Hebrew',
 'Hepburn',
 'Japanese',
 'Label',
 'Language',
 'Length',
 'Literally',
 'Mandarin',
 'McCune–Reischauer',
 'Music by',
 'Narrated by',
 'No. of episodes',
 'Opening theme',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Polish',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Revised Romanization',
 'Romanized',
 'Running time',
 'Russ

### Handle the Alternative Titles
*Plan - Now we can add in code to handle the alternative titles. The logic we need to implement follows:

- Make an empty dict to hold all of the alternative titles.
- Loop through a list of all alternative title keys:
    - Check if the current key exists in the movie object.
    - If so, remove the key-value pair and add to the alternative titles dict.
- After looping through every key, add the alternative titles dict to the movie object.

Try to implement the logic above in your "clean_movie" function on your own.

Hint: To remove a key-value pair from a dict in Python, use the "pop()" method.

*Execute - Step 1: Make an empty dict to hold all of the alternative titles.

In [26]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    return movie

Step 2: Loop through a list of all alternative title keys.

In [37]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:

        return movie

Step 2a: Check if the current key exists in the movie object.

In [39]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:

            return movie

Step 2b: If so, remove the key-value pair and add to the alternative titles dictionary.

In [40]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)


        return movie

Step 3: After looping through every key, add the alternative titles dict to the movie object.

In [41]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    return movie

We can make a list of cleaned movies with a list comprehension:

In [42]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

Set wiki_movies_df to be the DataFrame created from clean_movies, and print out a list of the columns.

In [43]:
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']